<a href="https://colab.research.google.com/github/Aljes2310/GIS-tasks-in-Python/blob/main/sentinel_api_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libreria a instalar



In [ ]:
pip install sentinelhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.4/244.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6085 sha256=52e8a73c859e0b2f7618d8567e20c454b95a6728835e243875b5a572734b9d6d
  Stored in directory: /root/.cache/pip/wheels/2f/a1/c8/543df0e8f5e824c3e92a432e32deb9cd89ae686095ee8cfcbe
Successfully built utm


Crear cuenta en https://www.sentinel-hub.com/trial y obtener credenciales en dashboard de usuario

In [ ]:
from sentinelhub import SHConfig

#mis credenciales
config = SHConfig(
    instance_id="39d7ff07-a7f6-44b3-9496-923c9b94aaaa",
    sh_client_id='384a00ae-40e2-4251-961f-5070a83370b7',
    sh_client_secret='W|}<E+gaqKY9r)VkB>DyTO@Ogde:C/[vnP@8BYmx'
)


if not config.sh_client_id or not config.sh_client_secret:
    print("Warning! To use Process API, please provide the credentials (OAuth client ID and client secret).")

config

SHConfig(
  instance_id='********************************aaaa',
  sh_client_id='********************************70b7',
  sh_client_secret='************************************BYmx',
  sh_base_url='https://services.sentinel-hub.com',
  sh_auth_base_url=None,
  sh_token_url='https://services.sentinel-hub.com/oauth/token',
  geopedia_wms_url='https://service.geopedia.world',
  geopedia_rest_url='https://www.geopedia.world/rest',
  aws_access_key_id='',
  aws_secret_access_key='',
  aws_session_token='',
  aws_metadata_url='https://roda.sentinel-hub.com',
  aws_s3_l1c_bucket='sentinel-s2-l1c',
  aws_s3_l2a_bucket='sentinel-s2-l2a',
  opensearch_url='http://opensearch.sentinel-hub.com/resto/api/collections/Sentinel2',
  max_wfs_records_per_query=100,
  max_opensearch_records_per_query=500,
  max_download_attempts=4,
  download_sleep_time=5.0,
  download_timeout_seconds=120.0,
  number_of_download_processes=1,
)

Importacion de Librerias

In [ ]:
import datetime
import os

import matplotlib.pyplot as plt
import numpy as np

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

# The following is not a package. It is a file utils.py which should be in the same folder as this notebook.
#from utils import plot_image

Definir el area de la imagen satelital

In [ ]:
#longitude and latitude coordinates of lower left and upper right corners)
betsiboka_coords_wgs84 = (-75.99937, -14.44883, -75.78805, -14.60921)

Definimos resolucion

In [ ]:
resolution = 10
betsiboka_bbox = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)
betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {betsiboka_size} pixels")

Image shape at 10 m resolution: (2270, 1783) pixels


Cargamos funcion

In [ ]:
evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""


Definimos Fecha, este scripts es para imagenes diarias

In [ ]:
start = datetime.datetime(2022, 1, 1)
end = datetime.datetime(2022, 1, 6)
n_chunks = 1 #numero de dias
tdelta = (end - start) / n_chunks
edges = [(start + i * tdelta).date().isoformat() for i in range(n_chunks)]
slots = [(edges[i], edges[i + 1]) for i in range(len(edges) - 1)]

for slot in slots:
    print(slot)

Cargamos funcion de descarga, se descarga imagenes como arrays

In [ ]:
def get_true_color_request(time_interval):
    return SentinelHubRequest(
        evalscript=evalscript_true_color,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C,
                time_interval=time_interval,
                mosaicking_order=MosaickingOrder.LEAST_CC,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
        bbox=betsiboka_bbox,
        size=betsiboka_size,
        config=config,
    )


# create a list of requests
list_of_requests = [get_true_color_request(slot) for slot in slots]
list_of_requests = [request.download_list[0] for request in list_of_requests]

# download data with multiple threads
data = SentinelHubDownloadClient(config=config).download(list_of_requests, max_threads=5)

Convertimos los arrays a png, primero definimos la carpeta donde queremos que se descarge los png

In [ ]:
from PIL import Image

for idx,x in enumerate(data):
  im = Image.fromarray(x)
  fecha = (start + datetime.timedelta(days=idx))
  im.save(f"{fecha.strftime('%Y-%m-%d')}.tif")